In [1]:
!pip install tensorflow==2.1.1 import_ipynb==0.1.3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=c106cd14620f9f07c8e7c54f77c407322fdea1eb8699edefeb124b216ce08dde
  Stored in directory: /home/jovyan/.cache/pip/wheels/06/7e/ad/1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2, MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
import import_ipynb
from codait_utils import *

importing Jupyter notebook from codait_utils.ipynb


In [3]:
unzip('.','data_small.zip')

In [4]:
!mv data_small data

In [5]:
batch_size = 32
img_height = 244
img_width = 244
input_shape = (img_width,img_height)
num_classes=2

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data/train',
  validation_split=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data/val',
  validation_split=None,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))



Found 88 files belonging to 2 classes.
Found 12 files belonging to 2 classes.


In [6]:
def my_net(model,freeze_layers=10,full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(2,activation='sigmoid')(x)
    model_final = Model(model.input,out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [7]:
#model = ResNet50V2(weights='imagenet',include_top=False)
#model = my_net(model)

In [8]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(244,244,3), alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=2,
    classifier_activation='softmax'
)
model = my_net(model)

9412608/9406464 [==============================] - 4s 0us/step


In [9]:
#model = tf.keras.applications.VGG16(
#    include_top=True, weights=None, input_tensor=None,
#    input_shape=(244, 244, 3), pooling=None, classes=2,
#    classifier_activation='softmax'
#)
#model = my_net(model)

In [10]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [11]:
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=2,
    validation_data=val_ds
)

Epoch 1/2
3/3 [==============================] - 13s 3s/step - loss: 0.9471 - accuracy: 0.4695 - val_loss: 0.7269 - val_accuracy: 0.5000
Epoch 2/2
3/3 [==============================] - 8s 3s/step - loss: 0.5012 - accuracy: 0.7578 - val_loss: 0.7440 - val_accuracy: 0.6667


In [12]:
!rm -Rf model

In [13]:
model.save('model')


INFO:tensorflow:Assets written to: model/assets


In [3]:
zipdir('model.zip', 'model')

adding model to model.zip


In [4]:
!rm -Rf model